In [1]:
import string
grid = [    ["_", "_", "_", "_", "Y"],
            ["R", "A", "_", "_", "_"],
            ["_", "_", "_", "_", "_"],
            ["_", "E", "_", "_", "_"],
            ["_", "_", "_", "_", "K"]
]
#Define Adjacency List
adjList = {
    1: [2,6], 2: [1, 3, 7], 3: [2, 4, 8], 4: [3, 5, 9], 5: [4, 10],
    6: [1, 7, 11], 7: [2, 6, 8, 12], 8: [3, 7, 9, 13], 9: [4, 8, 10, 14], 10: [5, 9, 15],
    11: [6,12,16], 12: [7, 11, 13, 17], 13: [8, 12, 14, 18], 14: [9, 13, 15, 19], 15: [10, 14, 20],
    16: [11,17,21], 17: [12, 16, 18, 22], 18: [13, 17, 19, 23], 19: [14, 18, 20, 24], 20: [15, 19, 25],
    21: [16,22], 22: [17, 21, 23], 23: [18, 22, 24], 24: [19, 23, 25], 25: [20, 24]
}

letters = set("ABCDEFGHIJKLMNOPQRSTUVWXY")

#Defines Domain List
import string

def find_domains(grid, adjList):
    # Create a dictionary to hold the domains for each cell
    domains = {}
    for i in range(1, 26):
        domains[i] = set(string.ascii_uppercase[:25])

    # Iterate over each cell in the grid and update its domain based on adjacent cells
    for var in range(1, 26):
        row = (var - 1) // 5
        col = (var - 1) % 5
        for adj in adjList[var]:
            adj_row = (adj - 1) // 5
            adj_col = (adj - 1) % 5
            adj_letter = grid[adj_row][adj_col]
            if adj_letter != '_':
                # Remove adjacent value and its neighbors from the domain of the current cell
                domains[var].discard(adj_letter)
                for neighbor in adjList[var]:
                    if neighbor != adj:
                        neighbor_row = (neighbor - 1) // 5
                        neighbor_col = (neighbor - 1) % 5
                        domains[var].discard(grid[neighbor_row][neighbor_col])

    # Update the domain of cells that already have a value assigned in the grid
    for var in range(1, 26):
        row = (var - 1) // 5
        col = (var - 1) % 5
        if grid[row][col] != '_':
            domains[var] = set(grid[row][col])

    return domains
#print(domainsList)

domainsList = find_domains(grid,adjList)
def position(grid,letter):
    pos = (1,-1)
    for i in range(5):
        for j in range(5):
            if grid[i][j] == letter:
                pos = (i,j)
                #pos = 5*i + j + 1
    return pos


def backtrack_search(grid, adjList, domainsList):
    # Check if the grid is complete
    if is_complete(grid):
        return grid

    # Select an unassigned variable
    var = select_unassigned_variable(grid, domainsList)
    
    # Try all possible values for the variable
    for value in domainsList[var]: #Iterates through each value (list) of the domain of var variable
        # Check if the value is consistent with the constraints
        #print(var)
        if is_consistent(var, value, grid, adjList):
            # Assign the value to the variable
            row = (var - 1) // 5
            col = (var - 1) % 5
            grid[row][col] = value
                     
            copied = domainsList
            # Update domains of adjacent variables
            adjacents = adjList.get(var)
            for adj in adjacents:
                if grid[(adj - 1) // 5][(adj - 1) % 5] == "_":
                    domain = domainsList[adj]
                    
                    if value in domain:
                        domainsList[adj].discard(value)
            
            # Recursively call the backtrack search function
            result = backtrack_search(grid, adjList, domainsList)
            if result is not None and verify_result(result,adjList):                
                return result

            # If the recursive call fails, unassign the variable and restore the domains of adjacent variables
            grid[row][col] = "_"
            domainsList = copied

    # If no value works, return None
    #print (var,domainsList[var])
    return None

def verify_result(grid, adjList):
    for i in range(1, 26):
        row = (i - 1) // 5
        col = (i - 1) % 5
        global count
        count = 0
        for adj in adjList[i]:
            adj_row = (adj - 1) // 5
            adj_col = (adj - 1) % 5
            #print(grid[row][col],grid[adj_row][adj_col])
            
            if abs(ord(grid[row][col]) - ord(grid[adj_row][adj_col])) == 1:
                count+=1
        if count<1:
            return False
    
    
    ''' 
    # First, find the positions of A and Y in the grid
    a_pos, y_pos = None, None
    for i in range(5):
        for j in range(5):
            if grid[i][j] == 'A':
                a_pos = (i, j)
            elif grid[i][j] == 'Y':
                y_pos = (i, j)
            if a_pos is not None and y_pos is not None:
                break
        if a_pos is not None and y_pos is not None:
            break
    
    # If either A or Y is missing, the grid is invalid
    if a_pos is None or y_pos is None:
        return False
    
    # Check if there is a path from A to Y using only adjacent letters
    # that are neighbors in the path from A to Y
    curr_pos = a_pos
    prev_letter = 'A'
    while curr_pos != y_pos:
        curr_letter = grid[curr_pos[0]][curr_pos[1]]
        neighbor_cells = [(curr_pos[0]-1, curr_pos[1]), (curr_pos[0]+1, curr_pos[1]), (curr_pos[0], curr_pos[1]-1), (curr_pos[0], curr_pos[1]+1)]
        
        if curr_letter!='A':
            neighbor_letters = [chr(ord(curr_letter) + 1), chr(ord(curr_letter) - 1) ]
        else:
            neighbor_letters = [chr(ord(curr_letter) + 1)]
        #print(neighbor_cells)
        #print(neighbor_letters)
        
        for neighbor in neighbor_letters:
            if position(grid,neighbor) not in neighbor_cells:
                return False
        curr_letter = neighbor_letters[0]
        curr_pos = position(grid,neighbor_letters[0])
    '''
    
    return True

def is_complete(grid):
    # Check if all cells in the grid are filled out
    for row in grid:
        if "_" in row:
            return False
    return True

def select_unassigned_variable(grid, domainsList):
    # Select the variable with the smallest domain (MRV)
    min_domain_size = float("inf") #minimum number of elements in domain
    min_domain_var = None #the variable with minimum domain size
    for i in range(1, 26):
        if grid[(i-1)//5][(i-1)%5] == "_":
            domain_size = len(domainsList[i])
            if domain_size < min_domain_size:
                min_domain_size = domain_size
                min_domain_var = i
    return min_domain_var

def is_consistent(cell, letter, grid, adjList):
    copy = [row[:] for row in grid]

    # Check if the letter is already assigned to another variable in the grid
    row = (cell - 1) // 5
    col = (cell - 1) % 5
    for i in range(5):
        for j in range(5):
            if grid[i][j] == letter:
                return False
    
    # Check if the letter is adjacent to any of its neighbors
    copy[row][col] = letter
    
    adjacents = adjList.get(cell) #The list of adjacents of the variable
    adjacent_found = False
    for adj in adjacents:
        adj_row = (adj - 1) // 5
        adj_col = (adj - 1) % 5
        #print(copy[row][col],copy[adj_row][adj_col])
        cell_ascii = ord(copy[row][col])
        adj_cel_ascii = ord(copy[adj_row][adj_col])
        if copy[adj_row][adj_col] != '_' and cell_ascii + 1 == adj_cel_ascii or cell_ascii - 1 == adj_cel_ascii:
        #if copy[adj_row][adj_col] != '_' and abs(ord(copy[row][col]) - ord(copy[adj_row][adj_col])) == 1:
            adjacent_found = True
            break
    if not adjacent_found:
        return False  
    
    # All constraints satisfied
    return True


res = backtrack_search(grid, adjList, domainsList)

for i in res:
    print(i)



['Q', 'P', 'O', 'N', 'Y']
['R', 'A', 'B', 'M', 'X']
['S', 'F', 'G', 'V', 'W']
['T', 'E', 'H', 'U', 'J']
['C', 'D', 'I', 'L', 'K']
